# <center>Introduction to viASP</center>


<!-- viASP is a visualization and interactive explanation tool for ASP programs and their stable models.

viASP allows you to explore the visualization in a variety of ways:

* Inspect iterations of recursive rules individually
* Explain the derivation of symbols with arrows
* Relax the constraints of unsatisfiable programs
* Toggle parts of the program
* Show the added symbols or all of them
* Inspect a single model
* Add `#show` statements on the fly
* Search models, signatures and rules.

![Example visualization](../docs/img/header.png) -->

In [ ]:
%%file sprinkler.lp
1{rain;sprinkler}1.
wet :- rain.
wet :- sprinkler.

To start visualizing ASP programs in a jupyter notebook, simply import viasp_jupyter. This will start the viASP backend.

In [ ]:
from viasp import Control
from viasp.server import startup

app = startup.run()


options = ['0']
programs = ['sprinkler.lp']

ctl = Control(options)
for path in programs:
    ctl.load(path)
ctl.ground([("base", [])])

with ctl.solve(yield_=True) as handle:
    for m in handle:
        print("Answer:\n{}".format(m))
        ctl.viasp.mark(m)
    print(handle.get())
ctl.viasp.show()
app.run()


<!-- The server supports modes `inline` with visualizations in the notebook, and `external` with visualizations on [http://localhost:8050/](http://localhost:8050/). If the notebook is running in Binder, the frontend is served on port 8050 through the Binder proxy. -->

Note that in Binder, it is necessary to reload the visualization using the `Click to Restart and Run` button. This takes up to 10 seconds.

# 2. The ``Control`` Proxy

If you already have a running clingo program and want to add viASP visualizations, you can use the proxy viASP ``Control`` object. This object is a wrapper around a clingo ``Control`` object and adds viASP functionality. 

In [ ]:
%%file hamiltonian.lp

node(1..4). start(1).
edge(1,2). edge(2,3). edge(2,4). edge(3,1).
edge(3,4). edge(4,1). edge(4,3).
{ hc(V,U) } :- edge(V,U).
reached(V) :- hc(S,V), start(S).
reached(V) :- reached(U), hc(U,V).
:- node(V), not reached(V).
:- hc(V,U), hc(V,W), U!=W.
:- hc(U,V), hc(W,V), U!=W.

In [ ]:
from clingo.application import clingo_main, Application
from viasp import Control
from viasp.server import startup
class App(Application):

    def main(self, ctl, files):
        ctl = Control(control=ctl, files=files)

        for path in files:
            ctl.load(path)
        ctl.ground([("base", [])])
        with ctl.solve(yield_=True) as handle:
            for m in handle:
                ctl.viasp.mark(m)
            print(handle.get())
        ctl.viasp.show()


app = startup.run()
if __name__ == "__main__":
    clingo_main(App(), ['0', 'hamiltonian.lp'])
    app.run()


# 3. The adaptable way: Using the viASP ``api`` independently of Clingo

If you already have a program and its stable models, you can use the viASP python ``api``. You can directly mark the stable models without solving the program again. 

In [ ]:
%%file sprinkler.lp

1{rain;sprinkler}1.
wet :- rain.
wet :- sprinkler.

In [ ]:
from viasp.viasp_jupyter import app 
import viasp

viasp.add_program_file('sprinkler.lp')
viasp.mark_from_string('rain.wet.')
viasp.mark_from_string('sprinkler.wet.')
viasp.show()
app.run_server(mode='inline')


Debugging information is available in the log file `viasp.log`.